In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from AnsatzLacs import *
from PurificationUtil import *
from scipy.linalg import sqrtm

In [ ]:
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 2 # size of the reference state (for purification)

xi_size = sys_size + 1
xi_ref_size = ref_size + 1

problem_seed = 42
swap_test_unitary = get_swap_unitary(sys_size)
swap_test_unitary_xi = get_swap_unitary(xi_size)

In [ ]:
def get_root_fidelity(p, q):
    """
    returns \sqrt{F(p, q)}
    """
    sqrt_p = sqrtm(p)
    sqrt_q = sqrtm(q)
    X = sqrt_p @ sqrt_q
    X_dag = np.conjugate(X).T
    return np.real(np.sqrt(np.power(np.trace(sqrtm(X_dag @ X)), 2)))

In [ ]:
# define rho and sigma
np.random.seed(problem_seed)

rho_ansatz = General(sys_size, sys_size, 2)
sigma_ansatz = General(sys_size, sys_size, 2)

rho_param = np.random.normal(size=rho_ansatz.get_parameter_num()) % (2 * np.pi)
sigma_param = np.random.normal(size=sigma_ansatz.get_parameter_num()) % (2 * np.pi)

rho_ansatz.set_params(rho_param)
sigma_ansatz.set_params(sigma_param)

rho = rho_ansatz.get_density_matrix()
sigma = sigma_ansatz.get_density_matrix()
expected = get_root_fidelity(rho, sigma)
print("Exact root fidelity:", expected)

In [ ]:
# # if you want to use cvxpy:
# n = 2**sys_size
# Y = cp.Variable(rho.shape, hermitian=True)
# Z = cp.Variable(rho.shape, hermitian=True)
# V = cp.Variable((2*n, 2*n))
# V = cp.bmat([[Y, np.eye(n)], [np.eye(n), Z]])

# constraints = [Y >> 0, Z >> 0, V >> 0]
# prob = cp.Problem(cp.Minimize(0.5*cp.real(cp.trace(Y @ rho) + cp.trace(Z @ sigma))),
#                   constraints)

# # Solve SDP
# prob.solve(verbose=False)
# # expected = -np.log2(prob.value)
# expected = prob.value
# print("Aprroximated root fidelity:", expected)

In [ ]:
# two different get trace functions for different size systems
def get_trace(ansatz1, ansatz2, shots):
    return get_trace_helper(swap_test_unitary, ansatz1, ansatz2, shots)

def get_trace2(ansatz1, ansatz2, shots):
    return get_trace_helper(swap_test_unitary_xi, ansatz1, ansatz2, shots)

In [ ]:
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation*delta)
    minus = fun(params - perturbation*delta)
    grad = (plus-minus)/(2*perturbation*delta)

    norm = np.linalg.norm(grad)
    if norm > 2:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
def get_trace_xi(x, shots):
    """
    Computes term like Tr[(|0><0|ω)ξ] and  Tr[(|1><1|τ)ξ]
    If x = True, computes Tr[(|1><1|τ)ξ]. Otherwise, computes Tr[(|0><0|ω)ξ]
    """
    if x:
        app = np.array([[0, 0], [0, 1]])
        ansatz = ansatz2
    else:
        app = np.array([[1, 0], [0, 0]])
        ansatz = ansatz1
        
    if shots is None:
        xi_dm = ansatz_xi.get_density_matrix()
        a = ansatz.get_density_matrix()
        return np.trace(np.real(np.kron(app, a)@xi_dm))
    else: 
        xi_dm = ansatz_xi.get_density_matrix()
        state2 = np.kron(app, ansatz.get_density_matrix())

        total_state = np.kron(xi_dm,state2)
        density_matrix = swap_test_unitary_xi @ total_state @ np.conjugate(swap_test_unitary_xi).T

        probabilities = np.real(density_matrix.diagonal())
        return calculate_trace(probabilities, xi_size, shots)

In [ ]:
pauli = "X" + "I"*(xi_size-1)
XI_unitary = get_expectation_unitary(pauli)

In [ ]:
def get_trace_pauli(shots):
    '''
    Compute Tr[(XI)ξ]
    '''
    if shots is None:
        xi_dm = ansatz_xi.get_density_matrix()
        X = np.array([[0, 1], [1, 0]])
        I = np.eye(2 ** (xi_size - 1))
        return np.real(np.trace(np.kron(X, I) @ xi_dm))
    
    else:
        xi_dm = ansatz_xi.get_density_matrix()
        density_matrix = XI_unitary @ xi_dm @ np.conjugate(XI_unitary).T
        probabilities = np.real(density_matrix.diagonal())

        rng = np.random.default_rng()
        counts = rng.multinomial(shots, np.fromiter(probabilities, dtype=float))
        probabilities_shots = counts / shots

        return np.dot(probabilities_shots, get_pauli_val(pauli))

In [ ]:
N = 2**(sys_size + 1)
def dual_cost_function_full(params):
    """
    Returns the value of the cost function
    params: [parameters for ω, parameters for τ, parameters for ξ, λ, μ, ν]
    """
    param1 = params[:dual_num_params]
    param2 = params[dual_num_params:2 * dual_num_params]
    param_xi = params[2 * dual_num_params:2 * dual_num_params + dual_num_xi_params]
    ansatz1.set_params(param1)
    ansatz2.set_params(param2)
    ansatz_xi.set_params(param_xi)
    lmbda = params[-3]
    mu = params[-2]
    nu = params[-1]
    
    exp_omega_rho = get_trace(ansatz1, rho_ansatz, num_shots)
    exp_tau_sigma = get_trace(ansatz2, sigma_ansatz, num_shots)
    termA = lmbda * exp_omega_rho + mu * exp_tau_sigma
    
    exp_omega_sqr = get_trace(ansatz1, ansatz1, num_shots)
    exp_tau_sqr = get_trace(ansatz2, ansatz2, num_shots)
    exp_xi_sqr = get_trace2(ansatz_xi, ansatz_xi, num_shots)
    termB = lmbda**2 * exp_omega_sqr + mu**2 * exp_tau_sqr + N + nu**2 * exp_xi_sqr
    
    exp_omega_xi = get_trace_xi(False, num_shots)
    exp_tau_xi = get_trace_xi(True, num_shots)
    exp_pauli_xi = get_trace_pauli(num_shots)
    termC = -2 * lmbda * nu * exp_omega_xi - 2 * mu * nu * exp_tau_xi - 2 * nu * exp_pauli_xi
    
    obj = termA / 2
    penal = termB + termC
    cost = obj + c * penal
    return cost, obj, penal

def dual_cost_function(params):
    cost, _, _ = dual_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def dual_callback(x, iteration):
    global c
    global lr
    global lr_wait
    
    fx, obj, penal = dual_cost_function_full(x)

    dual_iterations.append(iteration)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)

    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 5:
            m,_ = np.polyfit(dual_iterations[-500:], dual_loss[-500:], 1)
            if m > 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = None
max_iter = 200 * 1000

c = 5
lr_max = 1e-1
lr_min = 1e-3
perturbation = 1e-4

dual_num_layers = 3
dual_num_params = get_parameter_number("general", sys_size, ref_size, dual_num_layers)
dual_num_xi_params = get_parameter_number("general", xi_size, xi_ref_size, dual_num_layers)

In [ ]:
num_train = 3

iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

record = 0
for i in range(num_train):
    np.random.seed(i)
    lr = lr_max
    lr_wait = 0
    
    lmbda = 1
    mu = 1
    nu = 1

    ansatz1 = create_ansatz("general", sys_size, ref_size, dual_num_layers) # ω
    ansatz2 = create_ansatz("general", sys_size, ref_size, dual_num_layers) # τ
    ansatz_xi = create_ansatz("general", xi_size, xi_ref_size, dual_num_layers) # ξ
    
    # training loop
    print("Expected value:", expected)
    init_params = [*ansatz1.params, *ansatz2.params, *ansatz_xi.params, lmbda, mu, nu]
    init_cost, init_obj, init_penal = dual_cost_function_full(init_params)
     
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objectives = [init_obj]
    dual_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(dual_cost_function, init_params, get_gradient, max_iter, callback=dual_callback)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)
    iterations_dual_outer.append(dual_iterations)
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"Fidelity_Purification_dual_data/seed{problem_seed}"
        problem_name = "Fid"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of times worked: {record}")

# Primal Optimization

In [ ]:
omega_size = sys_size + 1
omega_ref_size = ref_size + 1
num_alpha = 16
swap_test_unitary_omega = get_swap_unitary(omega_size)

In [ ]:
def int_to_pauli_string(x):
    return ("" if x < 4 else int_to_pauli_string(x//4)) + 'IXYZ'[x % 4]

x = list(range(4**sys_size))
pauli_strings = []
for i in range(4**sys_size):
    string = int_to_pauli_string(x[i])
    string = (sys_size-len(string))*'I' + string
    pauli_strings.append(string)

print(pauli_strings)
Xpaulis = ['X' + s for s in pauli_strings]
Ypaulis = ['Y' + s for s in pauli_strings]
all_paulis = Xpaulis + Ypaulis
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in all_paulis}
prod_unitary_dict = {pauli_string: get_pauli_prod(pauli_string) for pauli_string in pauli_strings}

In [ ]:
rho_0 = np.kron(np.array([[1, 0], [0, 0]]), rho)
sigma_1 = np.kron(np.array([[0, 0], [0, 1]]), sigma)

In [ ]:
def get_trace_omega(rho, shots):
    """
    Calculate terms Tr[(|0><0|ρ)ω] or Tr[(|1><1|σ)ω]
    If rho = True, returns Tr[(|0><0|ρ)ω]
    Uf rho = False, returns Tr[(|1><1|σ)ω]
    """
    state1 = rho_0 if rho else sigma_1
    state_omega = ansatz_omega.get_density_matrix()
    # if noiseless simulation
    if shots is None:
        return np.real(np.trace(state1 @ state_omega))
    
    # with shots
    else:
        total_state = np.kron(state1, state_omega)
        density_matrix = swap_test_unitary_omega @ total_state @ np.conjugate(swap_test_unitary_omega).T

        probabilities = np.real(density_matrix.diagonal())
        return calculate_trace(probabilities, omega_size, shots)

In [ ]:
def get_X(alpha):
    res = np.zeros((2 ** sys_size, 2 ** sys_size), dtype=np.cdouble)
    for i in range(len(alpha)):
        res += alpha[i] * prod_unitary_dict[pauli_strings[i]]
    return res

def get_pauli_omega_trace(alpha, shots):
    """
    calculate terms Tr[(|1><0|X)ω]"""
    if shots is None:
        X = get_X(alpha)
        omega_state = ansatz_omega.get_density_matrix()
        XY = np.array([[0, 0], [1, 0]], dtype=np.cdouble)
        return np.real(np.trace(np.kron(XY, X) @ omega_state))

    else:
        tr_x_omega = np.zeros(num_alpha)
        tr_y_omega = np.zeros(num_alpha)
        for i in range(num_alpha):
            tr_x_omega[i] = get_exp_helper(expectation_unitary_dict, Xpaulis[i], ansatz_omega, shots)
            tr_y_omega[i] = get_exp_helper(expectation_unitary_dict, Ypaulis[i], ansatz_omega, shots)
        return np.real(np.dot(alpha, tr_x_omega - 1j * tr_y_omega) / 2)

In [ ]:
# Some precompute terms
rho_squared = get_trace(rho_ansatz, rho_ansatz, None)
sigma_squared = get_trace(sigma_ansatz, sigma_ansatz, None)

In [ ]:
N = 2**sys_size
def primal_cost_function_full(params):
    """
    params: in the form of [real alphas, complex alphas, omega_parameters, lambda]
    """
    real_alpha = params[:num_alpha]
    complex_alpha = np.multiply(params[num_alpha:2 * num_alpha], 1j)
    alpha = np.add(real_alpha, complex_alpha)
    omega_params = params[2 * num_alpha: 2 * num_alpha + primal_num_params]
    lmbda = params[-1]
    
    ansatz_omega.set_params(omega_params)
    
    termA = N * real_alpha[0]
    
    omega_squared = get_trace2(ansatz_omega, ansatz_omega, num_shots)
    trXX = N * (np.linalg.norm(alpha) ** 2)
    termB = rho_squared + sigma_squared + lmbda**2 * omega_squared + 2 * trXX
    
    tr_rho_omega = get_trace_omega(True, num_shots)
    tr_sigma_omega = get_trace_omega(False, num_shots)
    termC = -2 * lmbda * (tr_rho_omega + tr_sigma_omega + 2 * get_pauli_omega_trace(alpha, num_shots))
    
    obj = termA
    penal = termB + termC
    
    cost = obj - c * penal
    return cost, obj, penal

def primal_cost_function(params):
    cost, _, _ = primal_cost_function_full(params)
    return -cost

In [ ]:
print_every = 100
def primal_callback(x, iteration):
    global lr
    global lr_wait
    
    fx, obj, penal = primal_cost_function_full(x)

    primal_iterations.append(iteration)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)

    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 3:
            m,_ = np.polyfit(primal_iterations[-300:], primal_loss[-300:], 1)
            if m < 0:
                lr = max(lr / 2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = None
max_iter = 100 * 1000
lr_max = 1e-1
lr_min = 5e-4
c = 40
perturbation = 1e-4

primal_num_layers = 4
primal_num_params = get_parameter_number("general", omega_size, ref_size, primal_num_layers)

In [ ]:
num_train = 3

iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

record = 0
for i in range(num_train):
    lr = lr_max
    lr_wait = 0
        
    np.random.seed(i)
    lmbda = 1
#     alpha_init = np.ones(num_alpha) / num_alpha
#     alpha_init = np.random.normal(size=num_alpha)
    alpha_init = np.zeros(num_alpha)
    ansatz_omega = create_ansatz("general", omega_size, ref_size, primal_num_layers)

    # training loop
    print("Expected value:", expected)
    init_params = [*alpha_init, *alpha_init, *ansatz_omega.params, lmbda]
    init_cost, init_obj, init_penal = primal_cost_function_full(init_params)
    
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objectives = [init_obj]
    primal_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(primal_cost_function, init_params, get_gradient, max_iter, callback=primal_callback)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)
    iterations_primal_outer.append(primal_iterations)
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"Fidelity_Purification_primal_data/seed{problem_seed}"
        problem_name = "Fid"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of times worked: {record}")

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 500
window_size_dual = 500

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(-8, 3)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()